In [2]:
import re
import pandas as pd
from pathlib import Path
from datasets import load_from_disk

def get_question_type(question):
    """
    Bestimmt den Fragetyp anhand des ersten Wortes der Frage.
    - Wenn das erste Wort ein typisches Fragewort (z. B. What, Where, When, How, Why, Who, Whom, Which) ist,
      wird genau dieses (capitalized) zurückgegeben.
    - Wenn das erste Wort ein Hilfsverb ist (z. B. is, are, do, etc.), wird "Yes/No" zurückgegeben.
    - Andernfalls wird "Other" zurückgegeben.
    """
    q = question.strip().lower()
    match = re.match(r"(\w+)", q)
    if match:
        first_word = match.group(1)
        interrogatives = {"what", "where", "when", "how", "why", "whose", "whom", "which"}
        if first_word in interrogatives:
            return first_word.capitalize()
        auxiliaries = {"is", "are", "do", "does", "did", "can", "could", "would", "will", "have", "has", "had", "am"}
        if first_word in auxiliaries:
            return "Yes/No"
        return "Other"
    else:
        return "Unknown"

# Definiere die Pfade zu den Datensätzen
project_root = Path.cwd().parent
data_path_train = project_root / "data" / "train"
data_path_validation = project_root / "data" / "validation"
data_path_test = project_root / "data" / "test"

# Lade die Datensätze (angenommen, sie wurden bereits mit load_from_disk gespeichert)
dataset_train = load_from_disk(str(data_path_train))
dataset_val = load_from_disk(str(data_path_validation))
dataset_test = load_from_disk(str(data_path_test))

# Wandle die Datensätze in Pandas DataFrames um
df_train = dataset_train.to_pandas()
df_val = dataset_val.to_pandas()
df_test = dataset_test.to_pandas()

# Führe alle DataFrames zusammen
df_all = pd.concat([df_train], ignore_index=True)

# Bestimme den Fragetyp für jede Frage
df_all["question_type"] = df_all["question"].apply(get_question_type)

# Gruppiere nach Fragetyp und berechne Anzahl und prozentualen Anteil
summary = df_all.groupby("question_type")["question"].agg(["count"]).reset_index()
total = summary["count"].sum()
summary["percentage"] = (summary["count"] / total * 100).round(1)
summary = summary.sort_values("count", ascending=False)

print("Fragetypen im gesamten Datensatz:")
print(summary)


Fragetypen im gesamten Datensatz:
  question_type  count  percentage
7        Yes/No   9728        49.5
2          What   8040        40.9
4         Where   1315         6.7
0           How    427         2.2
6           Why     71         0.4
1         Other     38         0.2
3          When     30         0.2
5         Whose      5         0.0


In [3]:
import pandas as pd
from pathlib import Path
from datasets import load_from_disk
import re

# Funktion zur Fragetyp-Bestimmung (wie bei dir)
def get_question_type(question):
    q = question.strip().lower()
    match = re.match(r"(\w+)", q)
    if match:
        first_word = match.group(1)
        interrogatives = {"what", "where", "when", "how", "why", "whose", "whom", "which"}
        if first_word in interrogatives:
            return first_word.capitalize()
        auxiliaries = {"is", "are", "do", "does", "did", "can", "could", "would", "will", "have", "has", "had", "am"}
        if first_word in auxiliaries:
            return "Yes/No"
        return "Other"
    else:
        return "Unknown"

# Pfad zum Trainingsdatensatz
project_root = Path.cwd().parent
data_path_train = project_root / "data" / "train"

# Lade und konvertiere den Datensatz
dataset_train = load_from_disk(str(data_path_train))
df_train = dataset_train.to_pandas()

# Fragetyp hinzufügen
df_train["question_type"] = df_train["question"].apply(get_question_type)

# Ziel: 50 % der Daten, stratifiziert nach Frage-Typ
fraction = 0.5
samples = []

for typ, group in df_train.groupby("question_type"):
    n = int(len(group) * fraction)
    sampled = group.sample(n=n, random_state=42)
    samples.append(sampled)

# Alle Samples zusammenführen
subset_df = pd.concat(samples)
subset_df = subset_df.sample(frac=1, random_state=42)

# Indizes der ausgewählten Samples (aus Original-DataFrame)
selected_indices = subset_df.index.tolist()

# Ausgabe
print(f"Anzahl gewählter Instanzen: {len(selected_indices)}")
print("Ein paar Beispiel-Indizes:", selected_indices[:10])

# Optional: Verteilung zur Kontrolle anzeigen
verteilung_kontrolle = subset_df["question_type"].value_counts(normalize=True) * 100
print("\nVerteilung im Sample (in %):")
print(verteilung_kontrolle.sort_index())


Anzahl gewählter Instanzen: 9825
Ein paar Beispiel-Indizes: [8607, 6805, 2809, 17702, 11404, 5895, 2048, 9256, 18777, 12133]

Verteilung im Sample (in %):
question_type
How        2.167939
Other      0.193384
What      40.916031
When       0.152672
Where      6.687023
Whose      0.020356
Why        0.356234
Yes/No    49.506361
Name: proportion, dtype: float64


In [5]:
len(selected_indices)

9825

In [6]:
# Speicherort festlegen
output_path = project_root / "docs" / "train_indices_50percent.txt"

# Indizes als Textdatei speichern
with open(output_path, "w") as f:
    for idx in selected_indices:
        f.write(f"{idx}\n")

print(f"\nIndizes wurden unter {output_path} gespeichert.")



Indizes wurden unter C:\Users\Arbnor\Desktop\BA\BA\Bachelor_Arbeit_VQA\docs\train_indices_50percent.txt gespeichert.
